# AlphaGPT 全流程运行

## 1. 全局环境设置 (Global Settings)
初始化随机种子、配置数据路径、定义回测时间段（训练集、验证集、测试集）。
- **Seed**: 确保实验可复现。
- **Periods**: Train(2018-2023), Valid(2024), Test(2025)。

In [1]:
import os
import random
import numpy as np
import torch
import pandas as pd
from IPython.display import display

from run_pipeline import train_ashare, run_multi_backtest as pipeline_run_multi_backtest
from model_core.run_backtest import run_joinquant_backtest, backtest_summary_row
from model_core.signal_engine import compute_signal_matrix
from model_core.vm import StackVM
from model_core.ops import OPS_CONFIG

FEATURES = ["RET", "LIQ_SCORE", "PRESSURE", "FOMO", "DEV", "LOG_VOL"]
SYMBOL_MAP = {"ADD": "+", "SUB": "-", "MUL": "*", "DIV": "/"}

vm_tokens = StackVM()
NEG_OP_IDX = next(i for i, (name, _, _) in enumerate(OPS_CONFIG) if name == "NEG")
NEG_TOKEN = int(vm_tokens.feat_offset) + int(NEG_OP_IDX)

def _fmt_op(op_name, args):
    arity = len(args)
    if arity == 1:
        if op_name == "NEG":
            return f"(-{args[0]})"
        return f"{op_name}({args[0]})"
    if arity == 2:
        if op_name in SYMBOL_MAP:
            return f"({args[0]} {SYMBOL_MAP[op_name]} {args[1]})"
        return f"{op_name}({args[0]}, {args[1]})"
    return f"{op_name}({', '.join(args)})"

def decode_postfix(tokens):
    stack = []
    for t in tokens:
        t = int(t)
        if t < len(FEATURES):
            stack.append(FEATURES[t])
            continue
        op_idx = t - len(FEATURES)
        if op_idx < 0 or op_idx >= len(OPS_CONFIG):
            return "INVALID_POSTFIX"
        op_name, _, arity = OPS_CONFIG[op_idx]
        if len(stack) < arity:
            return "INVALID_POSTFIX"
        args = [stack.pop() for _ in range(arity)]
        args.reverse()
        stack.append(_fmt_op(op_name, args))
    if len(stack) != 1:
        return "INVALID_POSTFIX"
    return stack[0]

def decode_prefix(tokens):
    stack = []
    for t in reversed(tokens):
        t = int(t)
        if t < len(FEATURES):
            stack.append(FEATURES[t])
            continue
        op_idx = t - len(FEATURES)
        if op_idx < 0 or op_idx >= len(OPS_CONFIG):
            stack.append(f"UNK_{t}")
            continue
        op_name, _, arity = OPS_CONFIG[op_idx]
        if len(stack) < arity:
            missing = arity - len(stack)
            for _ in range(missing):
                stack.append(FEATURES[0])
        args = [stack.pop() for _ in range(arity)]
        stack.append(_fmt_op(op_name, args))
    return stack[-1] if stack else "INVALID_PREFIX"

def negate_formula(tokens, token_mode):
    tokens = [int(t) for t in tokens]
    if token_mode == "postfix":
        return tokens + [NEG_TOKEN]
    return [NEG_TOKEN] + tokens

def run_multi_backtest(formula, periods, token_mode, instruments, provider_uri):
    return pipeline_run_multi_backtest(
        formula=formula,
        periods=periods,
        instruments=instruments,
        provider_uri=provider_uri,
        backtest_mode="cross_validate",
        token_mode=token_mode,
    )


def analyze_formula(tokens, mode, vm):
    tokens = [int(t) for t in tokens]
    print(f"Original Tokens ({len(tokens)}): {tokens}")
    
    if mode == 'postfix':
        repaired = vm.repair_postfix(tokens)
        expr = decode_postfix(repaired)
    else:
        repaired = vm.repair_prefix(tokens)
        expr = decode_prefix(repaired)
        
    print(f"Repaired Tokens ({len(repaired)}): {repaired}")
    print(f"Repaired Formula: {expr}")
    return repaired, expr


In [ ]:
seed = None
seed = random.SystemRandom().randint(0, 2**32 - 1) if seed is None else int(seed)
os.environ["PYTHONHASHSEED"] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

provider_uri = "/Users/shuyan/.qlib/qlib_data/cn_data"
instruments = "csi300"

train_start, train_end = "2018-01-01", "2023-12-31"
valid_start, valid_end = "2024-01-01", "2024-12-31"
test_start, test_end = "2025-01-01", "2025-12-31"

train_steps = 1000
batch_size = 512
max_formula_len = 12

periods = [
    ("train", train_start, train_end),
    ("valid", valid_start, valid_end),
    ("test", test_start, test_end),
]

used_seed = seed


## 2. 后缀表达式模式 (Postfix Mode)
AlphaGPT 使用后缀表达式（Reverse Polish Notation）进行因子挖掘。
在此模式下，操作符位于操作数之后（例如 `A B +` 代表 `A + B`）。

In [3]:
formula_postfix_trained, strategy_path_postfix, weights_path_postfix = train_ashare(
    train_start=train_start,
    train_end=train_end,
    valid_start=valid_start,
    valid_end=valid_end,
    instruments=instruments,
    batch_size=batch_size,
    train_steps=train_steps,
    max_formula_len=max_formula_len,
    output_dir="outputs_postfix",
    provider_uri=provider_uri,
    seed=used_seed,
    token_mode="postfix",
)
formula_postfix_trained = [int(t) for t in formula_postfix_trained]


Seed: 328738204
Slicing loader: removing first 488 days (Buffer period) to restore speed...
Slicing loader: removing first 484 days (Buffer period) to restore speed...
🚀 Starting Alpha Mining...


100%|██████████| 1000/1000 [21:12<00:00,  1.27s/it, TrainRew=-5.000, ValidBest=3.614]          


✅ Training Completed. Best Valid Score: 3.6137


### 2.1 查看原始 Token (Raw Tokens)
模型生成的原始整数序列，代表特征索引和操作符索引。

In [4]:
formula_postfix_trained

[9, 6, 8, 6, 6, 7, 8, 15, 6, 2, 6, 12]

### 2.2 解码公式 (Decode Formula)
将 Token 序列解码为人类可读的数学公式形式。

In [5]:
analyze_formula(formula_postfix_trained, "postfix", vm_tokens)

Original Tokens (12): [9, 6, 8, 6, 6, 7, 8, 15, 6, 2, 6, 12]
Repaired Tokens (21): [0, 0, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 15, 0, 6, 2, 6, 12]
Repaired Formula: SIGN(((DECAY((((((((RET / RET) + RET) * RET) + RET) + RET) - RET) * RET)) + RET) + PRESSURE))


([0, 0, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 15, 0, 6, 2, 6, 12],
 'SIGN(((DECAY((((((((RET / RET) + RET) * RET) + RET) + RET) - RET) * RET)) + RET) + PRESSURE))')

### 2.3 原始公式回测 (Backtest Positive)
直接使用解码出的公式在 A 股数据上进行回测，计算 IC、ICIR、Rank IC 等指标。
- **Postfix Exec**: 使用后缀堆栈执行引擎计算。
- **Prefix Exec**: 尝试将其转换为前缀形式执行（用于交叉验证）。

In [6]:
summary_postfix_train_postfix_exec = run_multi_backtest(
    formula_postfix_trained,
    periods,
    token_mode="postfix",
    instruments=instruments,
    provider_uri=provider_uri,
)
summary_postfix_train_prefix_exec = run_multi_backtest(
    formula_postfix_trained,
    periods,
    token_mode="prefix",
    instruments=instruments,
    provider_uri=provider_uri,
)

print("postfix")
display(summary_postfix_train_postfix_exec)

print("prefix")
display(summary_postfix_train_prefix_exec)


postfix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-36.377004,-14.878803,-21.498201,-62.083441,-0.097847,both,一致
1,valid,-0.114964,14.683342,-14.798306,-28.329294,-0.520471,both,一致
2,test,21.266007,17.663217,3.602790,-11.648798,0.214224,both,一致


prefix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-11.836308,-14.878803,3.042495,-42.925722,0.006405,joinquant_like,不一致
1,train,-12.190048,-14.878803,2.688756,-43.252428,0.002961,qlib,不一致
2,valid,16.827981,14.683342,2.144639,-12.164538,0.067989,both,一致
3,test,10.285751,17.663217,-7.377466,-8.878392,-0.631031,both,一致


### 2.4 取反公式回测 (Backtest Negative)
如果因子的 IC 为负（负相关），通常通过取反（`NEG` 操作）将其转换为正向因子。
这里测试对公式取反后的表现。

In [7]:
formula_postfix_exec_neg = negate_formula(formula_postfix_trained, token_mode="postfix")
formula_prefix_exec_neg_from_postfix_tokens = negate_formula(formula_postfix_trained, token_mode="prefix")

summary_postfix_train_postfix_exec_neg = run_multi_backtest(
    formula_postfix_exec_neg,
    periods,
    token_mode="postfix",
    instruments=instruments,
    provider_uri=provider_uri,
)
summary_postfix_train_prefix_exec_neg = run_multi_backtest(
    formula_prefix_exec_neg_from_postfix_tokens,
    periods,
    token_mode="prefix",
    instruments=instruments,
    provider_uri=provider_uri,
)

print("postfix")
display(summary_postfix_train_postfix_exec_neg)

print("prefix")
display(summary_postfix_train_prefix_exec_neg)


postfix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-63.403204,-14.878803,-48.524400,-102.755076,-0.497826,joinquant_like,不一致
1,train,-37.863258,-14.878803,-22.984454,-84.506295,-0.193030,qlib,不一致
2,valid,25.321065,14.683342,10.637723,-14.009619,0.513289,both,一致
3,test,-2.360658,17.663217,-20.023875,-18.691599,-1.405413,both,一致


prefix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-22.413900,-14.878803,-7.535096,-39.774556,-0.124930,both,一致
1,valid,4.541812,14.683342,-10.141530,-26.426565,-0.600548,both,一致
2,test,0.761098,17.663217,-16.902118,-26.353098,-0.566319,both,一致


### 2.5 查看取反公式 (View Negated Formula)
显示取反后的公式结构。

In [8]:
print("Postfix (Neg) Formula Structure:")
analyze_formula(formula_postfix_exec_neg, "postfix", vm_tokens)
print("\nPrefix (Neg) Formula Structure (converted):")
analyze_formula(formula_prefix_exec_neg_from_postfix_tokens, "prefix", vm_tokens)

Postfix (Neg) Formula Structure:
Original Tokens (13): [9, 6, 8, 6, 6, 7, 8, 15, 6, 2, 6, 12, 10]
Repaired Tokens (22): [0, 0, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 15, 0, 6, 2, 6, 12, 10]
Repaired Formula: (-SIGN(((DECAY((((((((RET / RET) + RET) * RET) + RET) + RET) - RET) * RET)) + RET) + PRESSURE)))

Prefix (Neg) Formula Structure (converted):
Original Tokens (13): [10, 9, 6, 8, 6, 6, 7, 8, 15, 6, 2, 6, 12]
Repaired Tokens (22): [10, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 0, 15, 6, 2, 6, 0, 12, 0]
Repaired Formula: (-(RET / (RET + (RET * (RET + (RET + (RET - (RET * DECAY((PRESSURE + (RET + SIGN(RET))))))))))))


([10, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 0, 15, 6, 2, 6, 0, 12, 0],
 '(-(RET / (RET + (RET * (RET + (RET + (RET - (RET * DECAY((PRESSURE + (RET + SIGN(RET))))))))))))')

## 3. 前缀表达式模式 (Prefix Mode)
使用前缀表达式（Polish Notation）重新训练模型。
在此模式下，操作符位于操作数之前（例如 `+ A B`）。

In [9]:
formula_prefix_trained, strategy_path_prefix, weights_path_prefix = train_ashare(
    train_start=train_start,
    train_end=train_end,
    valid_start=valid_start,
    valid_end=valid_end,
    instruments=instruments,
    batch_size=batch_size,
    train_steps=train_steps,
    max_formula_len=max_formula_len,
    output_dir="outputs_prefix",
    provider_uri=provider_uri,
    seed=used_seed,
    token_mode="prefix",
)
formula_prefix_trained = [int(t) for t in formula_prefix_trained]


Seed: 328738204
Slicing loader: removing first 488 days (Buffer period) to restore speed...
Slicing loader: removing first 484 days (Buffer period) to restore speed...
🚀 Starting Alpha Mining...


100%|██████████| 1000/1000 [21:32<00:00,  1.29s/it, TrainRew=-5.000, ValidBest=4.044]          


✅ Training Completed. Best Valid Score: 4.0441


### 3.1 查看原始 Token (Raw Tokens)

In [10]:
formula_prefix_trained

[1, 17, 16, 12, 13, 7, 13, 6, 17, 9, 6, 3]

### 3.2 解码公式 (Decode Formula)

In [11]:
analyze_formula(formula_prefix_trained, "prefix", vm_tokens)

Original Tokens (12): [1, 17, 16, 12, 13, 7, 13, 6, 17, 9, 6, 3]
Repaired Tokens (21): [6, 1, 17, 16, 12, 13, 0, 0, 7, 0, 13, 0, 0, 6, 0, 17, 9, 0, 6, 0, 3]
Repaired Formula: (LIQ_SCORE + MAX3(DELAY1(SIGN(GATE(RET, RET, (RET - GATE(RET, RET, (RET + MAX3((RET / (RET + FOMO)))))))))))


([6, 1, 17, 16, 12, 13, 0, 0, 7, 0, 13, 0, 0, 6, 0, 17, 9, 0, 6, 0, 3],
 '(LIQ_SCORE + MAX3(DELAY1(SIGN(GATE(RET, RET, (RET - GATE(RET, RET, (RET + MAX3((RET / (RET + FOMO)))))))))))')

### 3.3 原始公式回测 (Backtest Positive)

In [12]:
summary_prefix_train_prefix_exec = run_multi_backtest(
    formula_prefix_trained,
    periods,
    token_mode="prefix",
    instruments=instruments,
    provider_uri=provider_uri,
)
summary_prefix_train_postfix_exec = run_multi_backtest(
    formula_prefix_trained,
    periods,
    token_mode="postfix",
    instruments=instruments,
    provider_uri=provider_uri,
)

print("prefix")
display(summary_prefix_train_prefix_exec)

print("postfix")
display(summary_prefix_train_postfix_exec)


prefix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-53.751320,-14.878803,-38.872516,-65.703751,-0.416766,joinquant_like,不一致
1,train,-54.146705,-14.878803,-39.267902,-66.265552,-0.422678,qlib,不一致
2,valid,11.353306,14.683342,-3.330036,-28.186410,-0.040311,both,一致
3,test,17.310780,17.663217,-0.352436,-19.957777,-0.077535,both,一致


postfix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-32.666593,-14.878803,-17.787790,-146.704704,0.055363,joinquant_like,不一致
1,train,-33.037114,-14.878803,-18.158310,-148.167132,0.053953,qlib,不一致
2,valid,7.496763,14.683342,-7.186579,-17.115559,-0.438604,both,一致
3,test,11.653026,17.663217,-6.010191,-13.247784,-0.384725,both,一致


### 3.4 取反公式回测 (Backtest Negative)

In [13]:
formula_prefix_exec_neg = negate_formula(formula_prefix_trained, token_mode="prefix")
formula_postfix_exec_neg_from_prefix_tokens = negate_formula(formula_prefix_trained, token_mode="postfix")

summary_prefix_train_prefix_exec_neg = run_multi_backtest(
    formula_prefix_exec_neg,
    periods,
    token_mode="prefix",
    instruments=instruments,
    provider_uri=provider_uri,
)
summary_prefix_train_postfix_exec_neg = run_multi_backtest(
    formula_postfix_exec_neg_from_prefix_tokens,
    periods,
    token_mode="postfix",
    instruments=instruments,
    provider_uri=provider_uri,
)

print("prefix")
display(summary_prefix_train_prefix_exec_neg)

print("postfix")
display(summary_prefix_train_postfix_exec_neg)


prefix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,-64.741927,-14.878803,-49.863123,-82.536241,-0.565121,joinquant_like,不一致
1,train,-66.053473,-14.878803,-51.174669,-85.872560,-0.578802,qlib,不一致
2,valid,11.353306,14.683342,-3.330036,-28.186410,-0.040311,both,一致
3,test,17.310780,17.663217,-0.352436,-19.957777,-0.077535,both,一致


postfix


,label,TotalReturn%,BenchReturn%,ExcessReturn%,MaxDD%,IR,Backtest,CrossCheck
0,train,4.660827,-14.878803,19.539630,-123.277267,0.219766,joinquant_like,不一致
1,train,-3.760130,-14.878803,11.118674,-142.426569,0.212282,qlib,不一致
2,valid,-3.370273,14.683342,-18.053615,-29.550519,-0.979722,both,一致
3,test,1.816187,17.663217,-15.847029,-17.308394,-1.060350,both,一致


### 3.5 查看取反公式 (View Negated Formula)

In [14]:
print("Prefix (Neg) Formula Structure:")
analyze_formula(formula_prefix_exec_neg, "prefix", vm_tokens)
print("\nPostfix (Neg) Formula Structure (converted):")
analyze_formula(formula_postfix_exec_neg_from_prefix_tokens, "postfix", vm_tokens)

Prefix (Neg) Formula Structure:
Original Tokens (13): [10, 1, 17, 16, 12, 13, 7, 13, 6, 17, 9, 6, 3]
Repaired Tokens (22): [6, 10, 1, 17, 16, 12, 13, 0, 0, 7, 0, 13, 0, 0, 6, 0, 17, 9, 0, 6, 0, 3]
Repaired Formula: ((-LIQ_SCORE) + MAX3(DELAY1(SIGN(GATE(RET, RET, (RET - GATE(RET, RET, (RET + MAX3((RET / (RET + FOMO)))))))))))

Postfix (Neg) Formula Structure (converted):
Original Tokens (13): [1, 17, 16, 12, 13, 7, 13, 6, 17, 9, 6, 3, 10]
Repaired Tokens (22): [1, 17, 16, 12, 0, 0, 13, 0, 7, 0, 0, 13, 0, 6, 17, 0, 9, 0, 6, 3, 10, 6]
Repaired Formula: (((MAX3((GATE((GATE(SIGN(DELAY1(MAX3(LIQ_SCORE))), RET, RET) - RET), RET, RET) + RET)) / RET) + RET) + (-FOMO))


([1, 17, 16, 12, 0, 0, 13, 0, 7, 0, 0, 13, 0, 6, 17, 0, 9, 0, 6, 3, 10, 6],
 '(((MAX3((GATE((GATE(SIGN(DELAY1(MAX3(LIQ_SCORE))), RET, RET) - RET), RET, RET) + RET)) / RET) + RET) + (-FOMO))')

## 4. 总结与交叉验证 (Summary & Cross-Validation)
对比 Postfix 和 Prefix 两种模式下生成的因子表现，并验证 **修复机制 (Repair Mechanism)** 的一致性。
通过 `StackVM` 检查是否可以将一种形式完美转换为另一种形式并保持结果一致。

## 4. 因子家谱与全面比武 (Factor Genealogy & Grand Tournament)

为了找出最强因子，我们不仅评估原生的训练结果，还通过**跨模式移植**和**方向取反**构建了全方位的候选池。
目前共有 **8 位候选者** 参与最终决战：

### 1. 嫡系正统 (Native Trained)
模型直接“思考”出来的原始结果，血统纯正。
- **Postfix (Pos)**: 后缀模式训练出的原生公式。
- **Prefix (Pos)**: 前缀模式训练出的原生公式。

### 2. 镜像取反 (Sign Reversal)
量化中常有“反向指标”现象（IC为负），因此我们将原生公式乘以 -1 再次评估。
- **Postfix (Neg)**: `Postfix (Pos) * -1`
- **Prefix (Neg)**: `Prefix (Pos) * -1`

### 3. 跨界移植 (Cross-Mode Translation)
利用 AlphaGPT 的双模特性，将一种模式学到的逻辑“强行翻译”成另一种模式执行，往往能挖掘出意外的超额收益（Out-of-Sample Alpha）。
- **Prefix -> Postfix (Pos/Neg)**: 将前缀逻辑翻译为后缀执行。
- **Postfix -> Prefix (Pos/Neg)**: 将后缀逻辑翻译为前缀执行（**新增**）。

In [15]:
# === Final Grand Summary (8 Candidates) ===
import pandas as pd
from IPython.display import display

def _pick_label_row(summary_df, label):
    df = summary_df[summary_df["label"] == label]
    if df.empty:
        return None
    if "Backtest" in df.columns:
        both = df[df["Backtest"] == "both"]
        if not both.empty:
            return both.iloc[0]
        jq = df[df["Backtest"] == "joinquant_like"]
        if not jq.empty:
            return jq.iloc[0]
        qlib = df[df["Backtest"] == "qlib"]
        if not qlib.empty:
            return qlib.iloc[0]
    return df.iloc[0]

def process_summary(summary_df, name, mode, formula_tokens):
    if summary_df is None or summary_df.empty:
        return None
    rows = []
    for label in ["train", "valid", "test"]:
        row = _pick_label_row(summary_df, label)
        if row is None:
            return None
        rows.append(dict(row))
    df = pd.DataFrame(rows)
    df.insert(0, "Name", name)
    df.insert(1, "Mode", mode)
    df["Formula_Obj"] = [formula_tokens] * len(df)
    return df

df_post_pos = process_summary(summary_postfix_train_postfix_exec, "Postfix (Pos)", "Postfix", formula_postfix_trained)
df_post_neg = process_summary(summary_postfix_train_postfix_exec_neg, "Postfix (Neg)", "Postfix", formula_postfix_exec_neg)
df_pre_pos = process_summary(summary_prefix_train_prefix_exec, "Prefix (Pos)", "Prefix", formula_prefix_trained)
df_pre_neg = process_summary(summary_prefix_train_prefix_exec_neg, "Prefix (Neg)", "Prefix", formula_prefix_exec_neg)
df_pre_post_pos = process_summary(summary_prefix_train_postfix_exec, "Prefix->Postfix (Pos)", "Postfix", formula_prefix_trained)
df_pre_post_neg = process_summary(summary_prefix_train_postfix_exec_neg, "Prefix->Postfix (Neg)", "Postfix", formula_postfix_exec_neg_from_prefix_tokens)

try:
    summary_postfix_train_prefix_exec = run_multi_backtest(
        formula_postfix_trained,
        periods,
        token_mode="prefix",
        instruments=instruments,
        provider_uri=provider_uri,
    )
    df_post_pre_pos = process_summary(summary_postfix_train_prefix_exec, "Postfix->Prefix (Pos)", "Prefix", formula_postfix_trained)
except Exception as e:
    print(f"Failed to run Postfix->Prefix (Pos): {e}")
    df_post_pre_pos = None

try:
    formula_postfix_exec_neg_as_prefix = negate_formula(formula_postfix_trained, token_mode="prefix")
    summary_postfix_train_prefix_exec_neg = run_multi_backtest(
        formula_postfix_exec_neg_as_prefix,
        periods,
        token_mode="prefix",
        instruments=instruments,
        provider_uri=provider_uri,
    )
    df_post_pre_neg = process_summary(summary_postfix_train_prefix_exec_neg, "Postfix->Prefix (Neg)", "Prefix", formula_postfix_exec_neg_as_prefix)
except Exception as e:
    print(f"Failed to run Postfix->Prefix (Neg): {e}")
    df_post_pre_neg = None

df_all = pd.concat(
    [
        df
        for df in [
            df_post_pos,
            df_post_neg,
            df_pre_pos,
            df_pre_neg,
            df_pre_post_pos,
            df_pre_post_neg,
            df_post_pre_pos,
            df_post_pre_neg,
        ]
        if df is not None and not df.empty
    ],
    ignore_index=True,
)

if df_all.empty:
    display(df_all)
else:
    df_test = df_all[df_all["label"] == "test"].copy()
    if df_test.empty:
        display(df_all)
    else:
        df_test = df_test.sort_values(by="ExcessReturn%", ascending=False)
        print("\n=== Final Tournament Results (Sorted by Test ExcessReturn%) ===")
        display(
            df_test[
                [
                    "Name",
                    "Mode",
                    "Backtest",
                    "CrossCheck",
                    "TotalReturn%",
                    "BenchReturn%",
                    "ExcessReturn%",
                    "IR",
                    "MaxDD%",
                ]
            ]
        )

        best = df_test.iloc[0]
        best_name = best["Name"]
        print(f"\n�� Best Performing Formula (Test Set): {best_name}")

        train_row = df_all[(df_all["Name"] == best_name) & (df_all["label"] == "train")]
        if not train_row.empty:
            train_excess = float(train_row.iloc[0].get("ExcessReturn%", 0.0))
            if train_excess < 0 and float(best.get("ExcessReturn%", 0.0)) > 0:
                print("⚠️ Insight: Reversal Detected (Train < 0, Test > 0).")
                print("   The factor direction flipped out-of-sample, likely capturing a regime shift.")

        tokens = best["Formula_Obj"]
        mode = str(best["Mode"]).lower()
        if mode == "postfix":
            repaired = vm_tokens.repair_postfix(tokens)
            expr = decode_postfix(repaired)
        else:
            repaired = vm_tokens.repair_prefix(tokens)
            expr = decode_prefix(repaired)

        print(f"Formula: {expr}")
        print(f"Repaired Tokens: {[int(x) for x in repaired]}")
        print(f"FORMULA_EXPR = \"{expr}\"")
        print(f"FORMULA_TOKENS = {[int(x) for x in repaired]}")
        print(f"Test ExcessReturn%: {float(best.get('ExcessReturn%', 0.0)):.4f}")
        print(f"Test TotalReturn%: {float(best.get('TotalReturn%', 0.0)):.4f}")
        print(f"Test BenchReturn%: {float(best.get('BenchReturn%', 0.0)):.4f}")
        print(f"Test IR: {float(best.get('IR', 0.0)):.6f}")
        print(f"Test MaxDD%: {float(best.get('MaxDD%', 0.0)):.4f}")



=== Final Tournament Results (Sorted by Test ExcessReturn%) ===


,Name,Mode,Backtest,CrossCheck,TotalReturn%,BenchReturn%,ExcessReturn%,IR,MaxDD%
2,Postfix (Pos),Postfix,both,一致,21.266007,17.663217,3.602790,0.214224,-11.648798
8,Prefix (Pos),Prefix,both,一致,17.310780,17.663217,-0.352436,-0.077535,-19.957777
11,Prefix (Neg),Prefix,both,一致,17.310780,17.663217,-0.352436,-0.077535,-19.957777
14,Prefix->Postfix (Pos),Postfix,both,一致,11.653026,17.663217,-6.010191,-0.384725,-13.247784
20,Postfix->Prefix (Pos),Prefix,both,一致,10.285751,17.663217,-7.377466,-0.631031,-8.878392
17,Prefix->Postfix (Neg),Postfix,both,一致,1.816187,17.663217,-15.847029,-1.060350,-17.308394
23,Postfix->Prefix (Neg),Prefix,both,一致,0.761098,17.663217,-16.902118,-0.566319,-26.353098
5,Postfix (Neg),Postfix,both,一致,-2.360658,17.663217,-20.023875,-1.405413,-18.691599



�� Best Performing Formula (Test Set): Postfix (Pos)
⚠️ Insight: Reversal Detected (Train < 0, Test > 0).
   The factor direction flipped out-of-sample, likely capturing a regime shift.
Formula: SIGN(((DECAY((((((((RET / RET) + RET) * RET) + RET) + RET) - RET) * RET)) + RET) + PRESSURE))
Repaired Tokens: [0, 0, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 15, 0, 6, 2, 6, 12]
FORMULA_EXPR = "SIGN(((DECAY((((((((RET / RET) + RET) * RET) + RET) + RET) - RET) * RET)) + RET) + PRESSURE))"
FORMULA_TOKENS = [0, 0, 9, 0, 6, 0, 8, 0, 6, 0, 6, 0, 7, 0, 8, 15, 0, 6, 2, 6, 12]
Test ExcessReturn%: 3.6028
Test TotalReturn%: 21.2660
Test BenchReturn%: 17.6632
Test IR: 0.214224
Test MaxDD%: -11.6488
